## Tabela Hash - colisão por enderecamento aberto

Uma tabela Hash é uma estrutura de dados de acesso direto, pois o índice do dado a ser acessado é conhecido.

O índice de acesso a cada valor é dado por uma função hash `h(k) -> i`, que recebe a chave K e retorna o índice para acessar o valor V. 

Este notebook apresenta a implementação de uma tabela Hash com tratamento de colisões, utilizando o **método de endereçamento aberto**, que utiliza a própria tabela para armazenar elementos com colisões, usando métodos de re-dispersão. Os três mais comuns são:
- linear
- quadrático
- hash duplo


O notebook https://github.com/Marcosddf/algoritmoseestruturasdedados/blob/master/tabela_hash_simples.ipynb possui uma tabela hash sem tratamento de colisões.

O notebook https://github.com/Marcosddf/algoritmoseestruturasdedados/blob/master/tabela_hash_encadeamento.ipynb implementa uma tabela com tratamento de colisões por encadeamento.

In [1]:
#include <iostream>
#include <stdlib.h> 
#include <string.h>
using namespace std;

#define M 19; //tamanho da tabela Hash
#define A_ASCII 65;

Identificação do tipo da dispersão escolhida.

In [2]:
enum Dispersao {
    LINEAR, QUADRATICA, HASH_DUPLO
}

O tamanho da tabela Hash depende do número de chaves existentes e da capacidade de armazenamento. 

In [3]:
int tamHash(){
    return M;
}

O **método da divisão** é bastante usado para implementar funções hash.

Dada uma tabela de tamanho **M**, a função hash será `h(k) -> k mod m`. M deverá ser preferencialmente um número primo, e potencias de 2 devem ser evitadas. 

Nesta implementação o M escolhido é 19.

Esta função pode ser trocada para implementar diferentes métodos.

In [4]:
int hash_divisao(int k){
    return (k % tamHash());
}

A função hash abaixo (`h(k) -> 1 + (k mod m-1)`) também implementa o hash da divisão, porém com uma variação do M, para ser usada no método de dispersão de hash duplo.

In [5]:
int hash_2_divisao(int k){
    return 1 + (k % (tamHash() - 2));
}

Dada uma tabela de tamanho **M**, a função hash usando dispersão linear será `h(k,i) -> (h'(k) + i) mod m`.

In [6]:
int hash_linear(int k, int i){
    return (hash_divisao(k) + i) % tamHash();
}

A função hash com disperção quadrática é implementada por: `h(k,i) = (h'(k) + c1*i + c2*i^2) mod m`

In [7]:
int hash_quadratico(int k, int i){
    int c1 = 2, c2 = 4;
    return (hash_divisao(k) + c1*i + c2*i*i ) % tamHash();
}

A função hash com disperção por hash duplo é implementada por: `h(k, i) = (h1(k) + i*h2(k)) mod m` 

In [8]:
int hash_duplo(int k, int i){
    int c1 = 2, c2 = 4;
    return (hash_divisao(k) + i*hash_2_divisao(k)) % tamHash();
}

Função para iniciar todos os elementos da tabela hash.

In [9]:
void inicia(char tabela[][10]){
    for (int i = 0; i < tamHash(); i ++)
        tabela[i][0] = '\0';
}

Função que converte um caracter ASCII para um índice numérico, que será a chave da string. Leva em consideração letras maiúsculas do alfabeto.

In [10]:
int char_to_chave(char chr){
    char c = toupper(chr);
    return c - A_ASCII;
}

Imprime os elementos da tabela Hash.

In [11]:
void imprime(char tabela[][10]){
    cout << endl << endl;
    for (int i = 0; i < tamHash(); i ++)
        cout << i << ':' << tabela[i] << ".";
}

Inclui um elemento na tabela Hash. Os elementos com colisão serão incluídos na próxima posição livre, de acordo com a técnica de dispersão escolhida: linear, quadrática ou hash duplo. 

Cada método de dispersão também poderia ser implementado em funções separadas.

In [12]:
void inclui (char tabelaHash[][10], const char *valor, Dispersao dispersao) {
    int i = 0, j;
    int chave = char_to_chave(valor[0]);
    do {
        if (dispersao == LINEAR)
            j = hash_linear(chave, i);
        else 
        if (dispersao == QUADRATICA)
            j = hash_quadratico(chave, i);
        else
            j = hash_duplo(chave, i);
        if (tabelaHash[j][0] == '\0') {
            strcpy(tabelaHash[j], valor);
            return;
        }        
        i = i + 1;
    } while (i < tamHash());
    cout << "\nTabela cheia, valor não incluído";
}

Busca um elemento na tabela Hash. Calcula o valor do índice usando a função Hash e retorna o elemento na posição da tabela.

In [13]:
char *busca (char tabelaHash[][10], char chave, Dispersao dispersao){
    int i = 0, j;
    char *retorno = (char *)malloc (sizeof (char)); retorno[0] = '\0';
    int chaveInt = char_to_chave(chave);
    do {
        if (dispersao == LINEAR) 
            j = hash_linear(chaveInt, i);
        else
        if (dispersao == QUADRATICA)
            j = hash_quadratico(chaveInt, i);
        else
            j = hash_duplo(chaveInt, i);
        if (tabelaHash[j][0] == chave)
            return tabelaHash[j];
        i = i + 1;
    } while (i < tamHash() && tabelaHash[j][0] != '\0');
    return retorno;
}

Inclui e busca elementos desta tabela Hash, com tratamento de colisões usando o **método de endereçamento aberto**, dispersão **linear** ou **quadrática**.

Não foi implementada função para exclusão. A exclusão de elementos poderia ser implementada apenas se um elemento excluído fique marcado como __excluido__, para que a busca funcione corretamente.

In [14]:
void iniciaprograma(){
    
    int t = tamHash();
    char tabelaHash[t][10];
    inicia(tabelaHash);
    
    inclui(tabelaHash, "Ariel", LINEAR);
    inclui(tabelaHash, "Lucas", LINEAR);
    inclui(tabelaHash, "Rafael", LINEAR);
    inclui(tabelaHash, "Pamela", LINEAR);
    inclui(tabelaHash, "Tamara", LINEAR);
    inclui(tabelaHash, "Val", LINEAR);
    inclui(tabelaHash, "Claus", LINEAR);
    inclui(tabelaHash, "Sonia", LINEAR);
    inclui(tabelaHash, "Sonia2", LINEAR);
    inclui(tabelaHash, "Bruno", LINEAR);
    
    imprime(tabelaHash);
    
    cout << endl;
    cout << "\n" << busca(tabelaHash, 'R', LINEAR);
    cout << "\n" << busca(tabelaHash, 'P', LINEAR);
    cout << "\n" << busca(tabelaHash, 'K', LINEAR);
    cout << "\n" << busca(tabelaHash, 'C', LINEAR);
    cout << "\n" << busca(tabelaHash, 'B', LINEAR);
        
    cout << "\n\n------------------------------------";
    cout << "\nMESMO CONJUNTO DE OPERAÇÕES, COM DISPERSÃO QUADRÁTICA";

    inicia(tabelaHash);
    
    inclui(tabelaHash, "Ariel", QUADRATICA);
    inclui(tabelaHash, "Lucas", QUADRATICA);
    inclui(tabelaHash, "Rafael", QUADRATICA);
    inclui(tabelaHash, "Pamela", QUADRATICA);
    inclui(tabelaHash, "Tamara", QUADRATICA);
    inclui(tabelaHash, "Val", QUADRATICA);
    inclui(tabelaHash, "Claus", QUADRATICA);
    inclui(tabelaHash, "Sonia", QUADRATICA);
    inclui(tabelaHash, "Sonia2", QUADRATICA);
    inclui(tabelaHash, "Bruno", QUADRATICA);
    
    imprime(tabelaHash);
    
    cout << endl;
    cout << "\n" << busca(tabelaHash, 'R', QUADRATICA);
    cout << "\n" << busca(tabelaHash, 'P', QUADRATICA);
    cout << "\n" << busca(tabelaHash, 'K', QUADRATICA);
    cout << "\n" << busca(tabelaHash, 'C', QUADRATICA);
    cout << "\n" << busca(tabelaHash, 'B', QUADRATICA);
    
cout << "\n\n------------------------------------";
    cout << "\nMESMO CONJUNTO DE OPERAÇÕES, COM HASH DUPLO";

    inicia(tabelaHash);
    
    inclui(tabelaHash, "Ariel", HASH_DUPLO);
    inclui(tabelaHash, "Lucas", HASH_DUPLO);
    inclui(tabelaHash, "Rafael", HASH_DUPLO);
    inclui(tabelaHash, "Pamela", HASH_DUPLO);
    inclui(tabelaHash, "Tamara", HASH_DUPLO);
    inclui(tabelaHash, "Val", HASH_DUPLO);
    inclui(tabelaHash, "Claus", HASH_DUPLO);
    inclui(tabelaHash, "Sonia", HASH_DUPLO);
    inclui(tabelaHash, "Sonia2", HASH_DUPLO);
    inclui(tabelaHash, "Bruno", HASH_DUPLO);
    
    imprime(tabelaHash);
    
    cout << endl;
    cout << "\n" << busca(tabelaHash, 'R', HASH_DUPLO);
    cout << "\n" << busca(tabelaHash, 'P', HASH_DUPLO);
    cout << "\n" << busca(tabelaHash, 'K', HASH_DUPLO);
    cout << "\n" << busca(tabelaHash, 'C', HASH_DUPLO);
    cout << "\n" << busca(tabelaHash, 'B', HASH_DUPLO);    
    
    
}

In [15]:
iniciaprograma();



0:Ariel.1:Tamara.2:Val.3:Claus.4:Sonia2.5:Bruno.6:.7:.8:.9:.10:.11:Lucas.12:.13:.14:.15:Pamela.16:.17:Rafael.18:Sonia.

Rafael
Pamela

Claus
Bruno

------------------------------------
MESMO CONJUNTO DE OPERAÇÕES, COM DISPERSÃO QUADRÁTICA

0:Ariel.1:Bruno.2:Val.3:.4:.5:Sonia2.6:Tamara.7:.8:Claus.9:.10:.11:Lucas.12:.13:.14:.15:Pamela.16:.17:Rafael.18:Sonia.

Rafael
Pamela

Claus
Bruno

------------------------------------
MESMO CONJUNTO DE OPERAÇÕES, COM HASH DUPLO

0:Ariel.1:Sonia2.2:Val.3:Tamara.4:.5:Claus.6:.7:Bruno.8:.9:.10:.11:Lucas.12:.13:.14:.15:Pamela.16:.17:Rafael.18:Sonia.

Rafael
Pamela

Claus
Bruno